In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Lambda
from sklearn.decomposition import PCA
import re

In [2]:
with open('CBOW.txt', 'r', encoding='utf-8') as file:
  data = file.read()
data

'The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n\nFurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n\nThe reproductive number – the number of secondary infections generated from one infected individual – is understood to be betwe

In [3]:
sentences = data.split(".")
sentences

['The speed of transmission is an important point of difference between the two viruses',
 ' Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus',
 ' The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days',
 ' This means that influenza can spread faster than COVID-19',
 ' \n\nFurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza',
 ' In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission',
 ' \n\nThe reproductive number – the number of secondary infections generated from one infected individual – 

In [4]:
clean_sentences = []

for sentence in sentences:
  if sentence=="":
    continue;
  sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
  sentence = re.sub(r'(?:^| )\w(?:$| )', ' ', sentence).strip()
  sentence = sentence.lower()
  clean_sentences.append(sentence)

clean_sentences

['the speed of transmission is an important point of difference between the two viruses',
 'influenza has shorter median incubation period the time from infection to appearance of symptoms and shorter serial interval the time between successive cases than covid 19 virus',
 'the serial interval for covid 19 virus is estimated to be 6 days while for influenza virus the serial interval is days',
 'this means that influenza can spread faster than covid 19',
 'further transmission in the first 5 days of illness or potentially pre symptomatic transmission transmission of the virus before the appearance of symptoms is major driver of transmission for influenza',
 'in contrast while we are learning that there are people who can shed covid 19 virus 24 48 hours prior to symptom onset at present this does not appear to be major driver of transmission',
 'the reproductive number the number of secondary infections generated from one infected individual is understood to be between and',
 'for covid 

In [5]:
corpus = clean_sentences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
print("Vector of words:")
print(sequences)

Vector of words:
[[1, 33, 2, 3, 9, 34, 35, 36, 2, 37, 12, 1, 38, 20], [4, 39, 21, 40, 41, 42, 1, 13, 22, 43, 10, 23, 2, 24, 11, 21, 14, 15, 1, 13, 12, 44, 45, 16, 5, 6, 7], [1, 14, 15, 8, 5, 6, 7, 9, 46, 10, 17, 47, 18, 25, 8, 4, 7, 1, 14, 15, 9, 18], [26, 48, 27, 4, 28, 49, 50, 16, 5, 6], [51, 3, 29, 1, 52, 53, 18, 2, 54, 55, 56, 57, 58, 3, 3, 2, 1, 7, 59, 1, 23, 2, 24, 9, 30, 31, 2, 3, 8, 4], [29, 60, 25, 61, 19, 62, 27, 63, 19, 64, 65, 28, 66, 5, 6, 7, 67, 68, 69, 70, 10, 71, 72, 73, 74, 26, 75, 76, 77, 10, 17, 30, 31, 2, 3], [1, 78, 32, 1, 32, 2, 79, 80, 81, 22, 82, 83, 84, 9, 85, 10, 17, 12, 11], [8, 5, 6, 7, 86, 16, 8, 4], [87, 88, 8, 89, 5, 6, 11, 4, 20, 19, 90, 91, 11, 13, 92, 93, 94, 95, 96, 97], []]


In [6]:
index_to_word_map = {}
word_to_index_map = {}

for index_1, sequence in enumerate(sequences):
  print(sequence)
  words_in_sentence = clean_sentences[index_1].split()
  for index_2, value in enumerate(sequence):
    index_to_word_map[value] = words_in_sentence[index_2]
    word_to_index_map[words_in_sentence[index_2]] = value

[1, 33, 2, 3, 9, 34, 35, 36, 2, 37, 12, 1, 38, 20]
[4, 39, 21, 40, 41, 42, 1, 13, 22, 43, 10, 23, 2, 24, 11, 21, 14, 15, 1, 13, 12, 44, 45, 16, 5, 6, 7]
[1, 14, 15, 8, 5, 6, 7, 9, 46, 10, 17, 47, 18, 25, 8, 4, 7, 1, 14, 15, 9, 18]
[26, 48, 27, 4, 28, 49, 50, 16, 5, 6]
[51, 3, 29, 1, 52, 53, 18, 2, 54, 55, 56, 57, 58, 3, 3, 2, 1, 7, 59, 1, 23, 2, 24, 9, 30, 31, 2, 3, 8, 4]
[29, 60, 25, 61, 19, 62, 27, 63, 19, 64, 65, 28, 66, 5, 6, 7, 67, 68, 69, 70, 10, 71, 72, 73, 74, 26, 75, 76, 77, 10, 17, 30, 31, 2, 3]
[1, 78, 32, 1, 32, 2, 79, 80, 81, 22, 82, 83, 84, 9, 85, 10, 17, 12, 11]
[8, 5, 6, 7, 86, 16, 8, 4]
[87, 88, 8, 89, 5, 6, 11, 4, 20, 19, 90, 91, 11, 13, 92, 93, 94, 95, 96, 97]
[]


In [7]:
print(index_to_word_map)
print('\n')
print(word_to_index_map)

{1: 'the', 33: 'speed', 2: 'of', 3: 'transmission', 9: 'is', 34: 'an', 35: 'important', 36: 'point', 37: 'difference', 12: 'between', 38: 'two', 20: 'viruses', 4: 'influenza', 39: 'has', 21: 'shorter', 40: 'median', 41: 'incubation', 42: 'period', 13: 'time', 22: 'from', 43: 'infection', 10: 'to', 23: 'appearance', 24: 'symptoms', 11: 'and', 14: 'serial', 15: 'interval', 44: 'successive', 45: 'cases', 16: 'than', 5: 'covid', 6: '19', 7: 'virus', 8: 'for', 46: 'estimated', 17: 'be', 47: '6', 18: 'days', 25: 'while', 26: 'this', 48: 'means', 27: 'that', 28: 'can', 49: 'spread', 50: 'faster', 51: 'further', 29: 'in', 52: 'first', 53: '5', 54: 'illness', 55: 'or', 56: 'potentially', 57: 'pre', 58: 'symptomatic', 59: 'before', 30: 'major', 31: 'driver', 60: 'contrast', 61: 'we', 19: 'are', 62: 'learning', 63: 'there', 64: 'people', 65: 'who', 66: 'shed', 67: '24', 68: '48', 69: 'hours', 70: 'prior', 71: 'symptom', 72: 'onset', 73: 'at', 74: 'present', 75: 'does', 76: 'not', 77: 'appear', 78

In [8]:
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 10
window_size = 2

contexts = []
targets = []

for sequence in sequences:
  for i in range(window_size, len(sequence)-window_size):
    context = sequence[i -window_size:i] + sequence[i + 1:i + window_size + 1]
    target = sequence[i]
    contexts.append(context)
    targets.append(target)

In [9]:
for i in range(5):
  words = []
  target = index_to_word_map.get(targets[i])
  for j in contexts[i]:
    words.append(index_to_word_map.get(j))
  print(words, '=>', target)

['the', 'speed', 'transmission', 'is'] => of
['speed', 'of', 'is', 'an'] => transmission
['of', 'transmission', 'an', 'important'] => is
['transmission', 'is', 'important', 'point'] => an
['is', 'an', 'point', 'of'] => important


In [10]:
X = np.array(contexts)
Y = np.array(targets)

In [11]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=2*window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(units=vocab_size, activation='softmax'))

In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=200)

Epoch 1/200
5/5 [==============================] - 3s 12ms/step - loss: 4.5846 - accuracy: 0.0268
Epoch 2/200
5/5 [==============================] - 0s 17ms/step - loss: 4.5733 - accuracy: 0.0537
Epoch 3/200
5/5 [==============================] - 0s 12ms/step - loss: 4.5570 - accuracy: 0.0537
Epoch 4/200
5/5 [==============================] - 0s 10ms/step - loss: 4.5307 - accuracy: 0.0537
Epoch 5/200
5/5 [==============================] - 0s 9ms/step - loss: 4.4849 - accuracy: 0.0537
Epoch 6/200
5/5 [==============================] - 0s 11ms/step - loss: 4.4113 - accuracy: 0.0537
Epoch 7/200
5/5 [==============================] - 0s 16ms/step - loss: 4.3246 - accuracy: 0.0537
Epoch 8/200
5/5 [==============================] - 0s 11ms/step - loss: 4.2584 - accuracy: 0.0537
Epoch 9/200
5/5 [==============================] - 0s 13ms/step - loss: 4.2487 - accuracy: 0.0537
Epoch 10/200
5/5 [==============================] - 0s 9ms/step - loss: 4.2005 - accuracy: 0.0738
Epoch 11/200
5/5 [===

In [13]:
embeddings = model.get_weights()[0]

pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

In [14]:
test_sentences = [
    'we are to study',
    'create programs direct processes'
]

for test_sentence in test_sentences:
  test_words = test_sentence.split(" ")
  print("Words: ", test_words)
  x_test = []
  for i in test_words:
    x_test.append(word_to_index_map.get(i))
  x_test = np.array([x_test])
  print("Indexs: ", x_test)
  test_predictions = model.predict(x_test)
  y_pred = np.argmax(test_predictions[0])
  print("Predictons: ",test_words, " => ", index_to_word_map.get(y_pred))

Words:  ['we', 'are', 'to', 'study']
Indexs:  [[61 19 10 None]]


ValueError: ignored